## Building Simple Filters and Queries
### SoQL Queries
SoQL statements are broken into “parameters” similar to clauses in SQL statements. Each clause can be expressed either directly as a URL parameter or as a SoQL statement. If a parameter is not specified, then the default is used. SoQL allows us to query the database in a SQL-like syntax, which means we can perform various operations like filtering `$where`, sorting `$order`, and aggregation `$group` directly at the API level. Make sure to add `&` after every statement. Querying datasets with simple equality filters is very easy with the SODA Consumer API. Simply use the column’s field name as your parameter and the content you want to filter for as its value. For instance:

#Each of these are SoQL statements starting with `$` 
```
$where=transit_timestamp >= '2023-12-31T00:00:00' AND transit_timestamp <= '2024-01-01T00:00:00' AND borough = 'Manhattan' AND station_complex = 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)' 

$order=transit_timestamp ASC 

$group=station_complex, transit_timestamp 

$select=station_complex, transit_timestamp, round(SUM(ridership),0) as total_ridership 
```

Note `round()` is used to remove trailing zero's for `ridership` column. For more info see here: https://dev.socrata.com/docs/transforms/round

Altogether, your final query will look like this: 

```https://data.ny.gov/resource/wujg-7c2s.geojson?$limit=1000000&$where=transit_timestamp >= '2023-12-31T00:00:00' AND transit_timestamp <= '2024-01-01T00:00:00' AND borough = 'Manhattan' AND station_complex='Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'&$order=transit_timestamp ASC&$group=station_complex, transit_timestamp&$select=station_complex, transit_timestamp, round(SUM(ridership),0) as total_ridership ```

## Lets Begin Querying!
Today we will be going over four different ways to analyze Subway Hourly Ridership Open Dataset using SoQL:
### 1. Peak Hour Analysis
By grouping and sorting data by station and hour, we can identify which stations are most crowded during New Year peak hours.

In [25]:
# Read in Subway data from API 
import pandas as pd
from io import StringIO
from data_utils import establish_nys_session
 

url = "https://data.ny.gov/resource/wujg-7c2s.csv?$limit=1000000&$where=transit_timestamp >= '2023-12-31T00:00:00' AND transit_timestamp <= '2024-01-01T00:00:00' AND borough = 'Manhattan' AND station_complex='Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'&$order=transit_timestamp ASC&$group=station_complex, transit_timestamp&$select=station_complex, transit_timestamp, round(SUM(ridership),0) as total_ridership " 
session = establish_nys_session()
response = session.get(url)

In [26]:
# Convert the CSV response to a DataFrame
data = StringIO(response.text)
df = pd.read_csv(data, header=0)
df

,station_complex,transit_timestamp,total_ridership
0,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",2023-12-31T00:00:00.000,3568
1,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",2023-12-31T01:00:00.000,1639
2,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",2023-12-31T02:00:00.000,852
3,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",2023-12-31T03:00:00.000,472
4,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",2023-12-31T04:00:00.000,364
5,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",2023-12-31T05:00:00.000,317
6,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",2023-12-31T06:00:00.000,779
7,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",2023-12-31T07:00:00.000,1239
8,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",2023-12-31T08:00:00.000,2335
9,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",2023-12-31T09:00:00.000,3689


### 2. Trend Analysis: 
Tracking sum of ridership over specific intervals can help observe trends in transit usage, which could be seasonal or event-driven. (4th of july 2022 vs 2023 ridership) one station at two years in a row. 

Here's a general approach to formulating such a query:

- Identify the Station ID or Name: Determine the unique identifier or name for the station of interest.
- Define the Date Ranges: Specify the date ranges for the days of interest in both years.
- Aggregate Ridership: Use SOQL functions to sum the ridership over your specified date ranges.
Here's is a conceptual example of how such a query might look:

For this Analysis lets use `$query` parameter:
The `$query` parameter allows you to combine multiple SoQL clauses together into a single parameter, for convenience. Similar to SQL, clauses must be specified in a specific order:
- SELECT
- WHERE
- ORDER BY
- GROUP BY
- LIMIT
- OFFSET

Note that unlike SQL, there is no **FROM** clause.

We can also use function `date_trunc_y(...)` which truncates a calendar date at the year threshold.

For example, you could combine `$select` and `$where` parameters together as follows:

```
$query=
SELECT
    date_trunc_y(transit_timestamp) AS year,
    station_complex,
    round(SUM(ridership),0) AS total_ridership
WHERE
    station_complex = 'Vernon Blvd-Jackson Av (7)' AND
    (
        (transit_timestamp >= '2022-07-04T00:00:00' AND transit_timestamp <= '2022-07-04T23:59:59') OR
        (transit_timestamp >= '2023-07-04T00:00:00' AND transit_timestamp <= '2023-07-04T23:59:59')
    )
GROUP BY
    year, station_complex
ORDER BY
    year ASC
```

In [31]:
import pandas as pd
from io import StringIO
from data_utils import establish_nys_session

url = """https://data.ny.gov/resource/wujg-7c2s.csv?$query=SELECT
    date_trunc_y(transit_timestamp) AS year,
    station_complex,
    round(SUM(ridership),0) AS total_ridership
WHERE
    station_complex = 'Vernon Blvd-Jackson Av (7)' AND
    (
        (transit_timestamp >= '2022-07-04T00:00:00' AND transit_timestamp <= '2022-07-04T23:59:59') OR
        (transit_timestamp >= '2023-07-04T00:00:00' AND transit_timestamp <= '2023-07-04T23:59:59')
    )
GROUP BY
    year, station_complex
ORDER BY
    year ASC
"""
session = establish_nys_session()
response = session.get(url)

In [32]:
# Convert the CSV response to a DataFrame
data = StringIO(response.text)
df = pd.read_csv(data, header=0)
df

,year,station_complex,total_ridership
0,2022-01-01T00:00:00.000,Vernon Blvd-Jackson Av (7),6599
1,2023-01-01T00:00:00.000,Vernon Blvd-Jackson Av (7),7718


Based on observation, there's an increase in total ridership from 6,599 in 2022 to 7,718 in 2023 at Vernon Blvd-Jackson Av (7). This represents a year-over-year growth in ridership.

### 3. Operational & Policy Decision Analayis

### Steps for Analysis and SOQL Query Construction:

Start by identifying all the unique fare categories for both MetroCard and OMNY present in the dataset. This preliminary analysis can guide the structure of your detailed query.

In [13]:
import pandas as pd
from io import StringIO
from data_utils import establish_nys_session

url = """https://data.ny.gov/resource/wujg-7c2s.csv?$query=SELECT fare_class_category
WHERE transit_timestamp >= '2023-12-31T00:00:00' AND transit_timestamp <= '2024-01-01T00:00:00'
GROUP BY fare_class_category
ORDER BY fare_class_category
"""
session = establish_nys_session()
response = session.get(url)

In [14]:
# Convert the CSV response to a DataFrame
data = StringIO(response.text)
df = pd.read_csv(data, header=0)
df

,fare_class_category
0,Metrocard - Fair Fare
1,Metrocard - Full Fare
2,Metrocard - Other
3,Metrocard - Seniors & Disability
4,Metrocard - Students
5,Metrocard - Unlimited 30-Day
6,Metrocard - Unlimited 7-Day
7,OMNY - Full Fare
8,OMNY - Other
9,OMNY - Seniors & Disability


### Compare Usage by Fare Category: 

Construct a SOQL query to compare ridership across different fare_class_category values for a specific time period. This query should separate MetroCard and OMNY usage, allowing for direct comparison.

```
$query=
SELECT fare_class_category, payment_method, round(SUM(ridership),0) AS total_ridership
WHERE transit_timestamp >= '2024-01-01T00:00:00' AND 
      transit_timestamp <= '2024-02-29T23:59:59'
GROUP BY fare_class_category, payment_method
ORDER BY fare_class_category, payment_method
```

In [36]:
import pandas as pd
from io import StringIO
from data_utils import establish_nys_session

url = """https://data.ny.gov/resource/wujg-7c2s.csv?$query=SELECT fare_class_category, payment_method, round(SUM(ridership),0) AS total_ridership
WHERE transit_timestamp >= '2024-03-01T00:00:00' AND 
      transit_timestamp <= '2024-03-15T23:59:59'
GROUP BY fare_class_category, payment_method
ORDER BY fare_class_category, payment_method
"""
session = establish_nys_session()
response = session.get(url)

In [37]:
# Convert the CSV response to a DataFrame
data = StringIO(response.text)
df = pd.read_csv(data, header=0)
df

,fare_class_category,payment_method,total_ridership
0,Metrocard - Fair Fare,metrocard,1170567
1,Metrocard - Full Fare,metrocard,5118283
2,Metrocard - Other,metrocard,1589365
3,Metrocard - Seniors & Disability,metrocard,1191097
4,Metrocard - Students,metrocard,1811753
5,Metrocard - Unlimited 30-Day,metrocard,3010755
6,Metrocard - Unlimited 7-Day,metrocard,2945407
7,OMNY - Full Fare,omny,17505281
8,OMNY - Other,omny,5533
9,OMNY - Seniors & Disability,omny,111413


Advocate for the introduction of OMNY fare categories that align with the MetroCard's, specifically targeting "Fair Fare", "Student", and "Unlimited" passes. This would not only ensure equitable access but could also encourage a shift from MetroCard to OMNY among broader user groups.